Deep Q- Learning
Deep Reinforcement Learning (심층강화학습)

-강화학습이란
강화학습 (Reinforcement Learning) 은 기계학습(Machine Learning)의 한 종류.
환경(입출력)에서 상호 작용하여 학습하는 Agent 제작.
Agent는 시행착오를 통해 학습을 하고, 많은 경험을 하게 됨.
(아기가 처음 걷는 법과 같은 학습 방법. 넘어지면서 여러 번 시도 끝에 걷는 것을 배움.)

-Deep Q Network (DQN)
13년도에 구글이 발표한 새로운 알고리즘.
Agent가 화면을 관찰하는 것만으로 어떻게 게임을 배울 수 있는지 보여줌.
딥러닝 + 강화학습 = Deep Reinforcement Learning
DQN 알고리즘의 신경망은 환경을 기반으로 최고의 동작을 수행하는데 사용됨. (State)
Q함수는 State를 기반으로 잠재적인 보상을 추정하는 데 사용됨. = Q (State, Action)
Q는 State와 Action을 기준으로 예상되는 미래의 값들을 계산하는 함수.



-CartPole
목표 : 움직이는 카트 위에 있는 폴의 균형을 맞추는 것.

Gym은 게임 환경과의 모든 상호작용을 단순화함. Agent의 두뇌에 초점을 맞춤.
 


In [ ]:
import keras
import gym

In [ ]:
# INPUT
# action은 0 또는 1

# OUTPUT
# 다음 State, 보상, 정보 : 우리가 무엇을 위해서 학습하는지에 대한 것
# done : 게임이 끝났는지 아닌지에 대한 boolean 타입의 값
next_state, reward, done, info = gym.Env.step(action)

#신경망만들기
-신경망이란
데이터 쌍(입력 및 출력 데이터)를 기반으로 학습. 특정 유형의 패턴 감지, 또 다른 입력 데이터를 기반으로 출력을 예측하는 알고리즘.

4개의 정보를 받는 입력 레이어, 3개의 히든레이어, 게임 버튼이 2개(0과 1)이므로 출력 레이어 2개.


In [ ]:
# Deep Q Learning 을 위한 신경망 만들기
# Sequential() 은 레이어를 쌓아줍니다.
model = Sequential()
# ‘Dense’ 는 신경망의 기본 form 입니다.
# State 의 입력 사이즈는 size(4) 이고, 히든 레이어는 24개의 노드로 이루어집니다.
model.add(Dense(24, input_dim=self.state_size, activation=’relu’))
# 24개의 노드를 가진 히든 레이어
model.add(Dense(24, activation=’relu’))
# 출력 레이어 : 2개의 노드 (left, right)
model.add(Dense(self.action_size, activation=’linear’))
# 정보를 토대로 모델을 만듭니다.
model.compile(loss=’mse’, optimizer=Adam(lr=self.learning_rate))

#신경망 학습

모델에 데이터를 입력해야함.
케라스는 입력 및 출력 쌍을 모델에 공급하는 `fit()`을 제공함.
그런 다음 모델은 이러한 데이터를 기반으로 학습되고, 입력을 기준으로 출력을 추정함.

이 학습 과정에서 신경망이 `state`로부터 보상을 예측할 수 있게함.


In [ ]:
model.fit(state, reward_value, epochs=1, verbose=0)

#예측
모델 학습 후, 입력 데이터로부터 출력(결과)을 예측할 수 있음.
`predict()` 함수는 모델이 훈련된 데이터를 기반으로 현재 `state`의 보상을 예측함.

In [ ]:
prediction = model.predict(state)

#전체코드


In [ ]:
import gym
import random
import os
import numpy as np
from collections      import deque
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam

class Agent():
    def __init__(self, state_size, action_size):
        self.weight_backup      = "cartpole_weight.h5"
        self.state_size         = state_size
        self.action_size        = action_size
        self.memory             = deque(maxlen=2000)
        self.learning_rate      = 0.001
        self.gamma              = 0.95
        self.exploration_rate   = 1.0
        self.exploration_min    = 0.01
        self.exploration_decay  = 0.995
        self.brain              = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))

        if os.path.isfile(self.weight_backup):
            model.load_weights(self.weight_backup)
            self.exploration_rate = self.exploration_min
        return model

    def save_model(self):
            self.brain.save(self.weight_backup)

    def act(self, state):
        if np.random.rand() <= self.exploration_rate:
            return random.randrange(self.action_size)
        act_values = self.brain.predict(state)
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, sample_batch_size):
        if len(self.memory) < sample_batch_size:
            return
        sample_batch = random.sample(self.memory, sample_batch_size)
        for state, action, reward, next_state, done in sample_batch:
            target = reward
            if not done:
              target = reward + self.gamma * np.amax(self.brain.predict(next_state)[0])
            target_f = self.brain.predict(state)
            target_f[0][action] = target
            self.brain.fit(state, target_f, epochs=1, verbose=0)
        if self.exploration_rate > self.exploration_min:
            self.exploration_rate *= self.exploration_decay

class CartPole:
    def __init__(self):
        self.sample_batch_size = 32
        self.episodes          = 10000
        self.env               = gym.make('CartPole-v1')

        self.state_size        = self.env.observation_space.shape[0]
        self.action_size       = self.env.action_space.n
        self.agent             = Agent(self.state_size, self.action_size)


    def run(self):
        try:
            for index_episode in range(self.episodes):
                state = self.env.reset()
                state = np.reshape(state, [1, self.state_size])

                done = False
                index = 0
                while not done:
                    self.env.render()

                    action = self.agent.act(state)

                    next_state, reward, done, _ = self.env.step(action)
                    next_state = np.reshape(next_state, [1, self.state_size])
                    self.agent.remember(state, action, reward, next_state, done)
                    state = next_state
                    index += 1
                print("Episode {}# Score: {}".format(index_episode, index + 1))
                self.agent.replay(self.sample_batch_size)
        finally:
            self.agent.save_model()

if __name__ == "__main__":
    cartpole = CartPole()
    cartpole.run()